### Practice/HW6 (10pt): Large Language Models and Their Implications
<!-- ![img](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F4470ce74-e595-4750-92a5-5f21f040df6d_577x432.jpeg) -->
![img](https://i.imgur.com/QGYa2J8.jpeg)

In this notebook, you're gonna play with some of the largest language models on the Internet.

_Based on works of: Tim Dettmers, Ruslan Svirschevsky, Artem Chumachenko, Younes Belkada, Felix Marty, Yulian Gilyazev, Gosha Zolotov, Andrey Ishutin,  Elena Volf, Artemiy Vishnyakov, Svetlana Shirokovskih. [Source notebook](https://github.com/yandexdataschool/nlp_course/tree/2024/week06_llm)

### Part 1: prompt engineering (4 points total)

In the assignment, we'll use public APIs that host the 100B+ models for inference. Your task is to prompt-engineer the model into solving a few tasks for you.


__Which API?__ You are free to use any publicly available API for general LM -- as long as it's __not a chat assistant__. So, gpt 3.5 is fine, but chatGPT is not. Here's a few options:

- BLOOM API - [bigscience/bloom](https://huggingface.co/bigscience/bloom) (currently not supported)
- OpenAI API - [openai.com/api](https://openai.com/api/)
- Mixtral API - https://huggingface.co/mistralai/Mixtral-8x7B-v0.1
- AI21 Jurrasic API - [ai21.com](https://www.ai21.com/blog/announcing-ai21-studio-and-jurassic-1)

These APIs may require you to create a (free) account on their platform. Please note that some APIs also have paid subscriptions. __You do not need to pay them__, this assignment was designed to be solved using free-tier subscriptions. If no APIs work for you, you can also solve these tasks with the 6.7B model that you will find later in this notebook - but this will make the tasks somewhat harder.

__Quests:__ you will need to solve 4 problems. For each one, please attach a short __description__ of your solution and the output from the API you use. _[If you use python APIs, show your python code with outputs]_

__Example:__ Tony is talking to Darth Vader ([BLOOM API](https://huggingface.co/bigscience/bloom)). Black text is written manually, blue text is generated.
<hr>

![img](https://i.imgur.com/a1QhKF7.png)
<hr>

__It is fine to roll back a few times,__ e.g. in the example above, the model first generated Vader lines twice in a row, and we rolled that back. However, if you need more than 1-2 rollbacks per session, you should probably try a different prompt.

__Task 1 (0.5 pt):__ arange a conversation between any two of the following:

- a celebrity or politician of your choice
- any fictional character (except Darth Vader)
- yourself

Compare two setups: a) you prompt with character names only b) you supply additional information (see example).

In [2]:
import os

import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()



MODEL = genai.GenerativeModel('gemini-1.5-flash')
api_key = os.getenv("GOOGLEAPIKEY")
genai.configure(api_key=api_key)


# Example characters
character1 = "Elon Musk"
character2 = "Sherlock Holmes"


In [3]:
prompt_A = f"{character1} and {character2} are having a conversation. Simulate their answers\n\n{character1}: "
conversation_A = MODEL.generate_content(prompt_A).text.strip()
print("### Setup A: Names Only ###\n", conversation_A)

DefaultCredentialsError: 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.

In [21]:
prompt_B = f"{character1} is a billionaire entrepreneur and CEO of SpaceX and Tesla. {character2} is a fictional detective known for his keen observation and deduction skills. Simulate a conversation between the two.\n\n {character1}: "
conversation_B = MODEL.generate_content(prompt_B).text.strip()
print("\n### Setup B: With extra information ###\n", conversation_B)




### Setup B: With extra information ###
 Elon Musk:  Holmes, fascinating to finally meet you.  I've always admired your… *method*.  A purely logical approach to seemingly impossible problems.  Reminds me a bit of software debugging, actually.  Find the bug, isolate the variable, solve the equation.  Except, your "bugs" are usually far more… *human*.

Sherlock Holmes: (Eyes Musk keenly, a slight smile playing on his lips)  Indeed, Mr. Musk.  And your "equations," as you put it, involve the complexities of interstellar travel and sustainable energy.  A different scale, certainly, but the underlying principles remain the same: observation, deduction, and a healthy dose of intuition.  Though your intuition seems to involve rather more risk capital than mine.

Elon Musk: (Chuckles)  Risk is inherent in innovation, Holmes.  Stagnation is the real killer.  Take Mars colonization, for instance.  The odds are long, the challenges immense… but the potential reward… well, it's rather hard to qua

__Please choose task 2a or 2b (0.5 pt)__ depending on your model (you can do both, but you will be awarded points for one of these two tasks).

__Task 2a: (for BLOOM or other multilingual model)__ zero-shot translation. Take the first verse of [Edgar Allan Poe's "Raven"](https://www.poetryfoundation.org/poems/48860/the-raven) and __translate it into French.__ (You are free to use any other text of at least the same size)

Original text: ```
Once upon a midnight dreary, while I pondered, weak and weary,
Over many a quaint and curious volume of forgotten lore—
    While I nodded, nearly napping, suddenly there came a tapping,
As of some one gently rapping, rapping at my chamber door.
“’Tis some visitor,” I muttered, “tapping at my chamber door—
            Only this and nothing more.” ```

Verify your translation by converting french back into english using a public machine translation service.


__Task 2b: (non-BLOOM)__ toxicity classification for [SetFit/toxic_conversations](https://huggingface.co/datasets/SetFit/toxic_conversations). Make the model solve binary classification (toxic vs not toxic) in the few shot mode. For few-shot examples, use 2-3 toxic and 2-3 non-toxic non-toxic examples. Measure accuracy on at least 25 samples. You may need to try several different prompts before you find the one that works.

In [22]:
text = '''Once upon a midnight dreary, while I pondered, weak and weary,
Over many a quaint and curious volume of forgotten lore—
    While I nodded, nearly napping, suddenly there came a tapping,
As of some one gently rapping, rapping at my chamber door.
“’Tis some visitor,” I muttered, “tapping at my chamber door—
            Only this and nothing more.'''

prompt_C = f"{text} \n\nTranslate the text into French."
translated_output = MODEL.generate_content(prompt_C).text.strip()

print("\n### Convert to french: ###\n", translated_output)


### Convert to french: ###
 Un soir d'hiver, sombre et froid, tandis que je méditai, faible et las,
Sur bien des volumes étranges et curieux d'un savoir oublié —
    Alors que je somnolais, presque endormi, soudain un frappant se fit entendre,
Comme si quelqu'un frappait doucement, frappant à ma porte de chambre.
“C'est un visiteur,” murmurai-je, “qui frappe à ma porte de chambre —
            Rien que cela et rien de plus.”


In [ ]:
translated_output = """Un soir d'hiver, sombre et froid, tandis que je méditai, faible et las,
Sur bien des volumes étranges et curieux d'un savoir oublié —
    Alors que je somnolais, presque endormi, soudain un frappant se fit entendre,
Comme si quelqu'un frappait doucement, frappant à ma porte de chambre.
“C'est un visiteur,” murmurai-je, “qui frappe à ma porte de chambre —
            Rien que cela et rien de plus."""

google_translated_output = """One winter evening, dark and cold, while I pondered, weak and weary,
Over many strange and curious volumes of forgotten lore—
As I dozed, almost asleep, suddenly a knocking was heard,
As if someone were gently rapping, knocking at my chamber door.
“It is a visitor,” I murmured, “who is knocking at my chamber door—
Nothing but that and nothing more."""



__Task 3 (0.5 pt):__ create a prompt and few-shot examples tha make the model __change the gender pronouns__ of the main actor in a given sentence in any direction of your choice. E.g. the doctor took off _his_ mask <-> the doctor took of _her_ mask.


In [23]:
prompt = """Change the gender pronouns in these sentences. Keep all other words the same.

Original: The doctor took off his mask after the surgery.
Changed: The doctor took off her mask after the surgery.

Original: She picked up her daughter from school.
Changed: He picked up his daughter from school.

Original: The nurse completed his rounds quickly.
Changed: The nurse completed her rounds quickly.

Original: The mechanic adjusted his tools before starting work.
Changed:"""

# Call the model
response = MODEL.generate_content(prompt).text.strip()
print("Response:", response)

# Test with another example
prompt2 = prompt + "\n\nOriginal: The teacher graded his papers over the weekend.\nChanged:"
response2 = MODEL.generate_content(prompt2).text.strip()
print("\nResponse 2:", response2)

Response: The mechanic adjusted her tools before starting work.

Response 2: Original: The mechanic adjusted his tools before starting work.
Changed: The mechanic adjusted her tools before starting work.

Original: The teacher graded his papers over the weekend.
Changed: The teacher graded her papers over the weekend.


__Task 4 (0.5 pt):__ write a prompt and supply examples such that the model would __convert imperial units to metric units__ (miles -> kilometers; mph -> kph). More specifically, the model should rewrite a given sentence and replace all imperial units with their metric equivalents. After it works with basic distances and speed, try to find complicated examples where it does *not* work.

Please note that 1 mile is not equal to 1 km :)

In [19]:
prompt = """Convert all imperial measurements to metric in these sentences. Keep all other words exactly the same.

Original: The car was traveling at 60 mph on the highway.
Converted: The car was traveling at 97 kph on the highway.

Original: The hiking trail is 2.5 miles long.
Converted: The hiking trail is 4 kilometers long.

Original: My house is 1500 feet from the station.
Converted: My house is 457 meters from the station.

Original: The truck needs to deliver supplies 120 miles away.
Converted:"""

# Test basic conversion
response = MODEL.generate_content(prompt).text.strip()
print("Response:", response)

# Test with more complex example
complex_prompt = prompt + "\n\nOriginal: The car gets 35 mpg and can travel 300 miles on a full tank at 75 mph.\nConverted:"
complex_response = MODEL.generate_content(complex_prompt).text.strip()
print("\nComplex Response:", complex_response)


hard_prompt = """Convert all imperial measurements to metric in these sentences. Keep all other words exactly the same.

Original: The engine gets 25 mpg at 65 mph with a curb weight of 4000 lbs and 18-inch wheels.
Converted: """

response = MODEL.generate_content(hard_prompt).text.strip()
print("Complex case (multiple units):", response)


NameError: name 'MODEL' is not defined

### Part 2: local inference

Now, let's try and load the strongest model that can fit a typical Colab GPU (T4 with 16 GB as of spring 2023).

Our best candidates are the smaller versions of the best performing open source models:
- 8 Bn parameters version of LLaMA
- 7 Bn parameters version of [LLaMA](https://arxiv.org/pdf/2302.13971.pdf) - best for spring 2023, released by Facebook
- 7 Bn parameters version of [Falcon](https://falconllm.tii.ae) - close competitor to Llama, released in May 2023 by [Technology Innovation Institute of UAE](https://www.tii.ae).
- 6.7 Bn parameters version of [OPT](https://arxiv.org/abs/2205.01068) - top choice in this nomination in 2022, released by Facebook.

Beware: while these models are smaller than the ones in API, they're still over 60x larger than the BERT we played with last time. The code below will *just barely* fit into memory, so make sure you don't have anything else loaded. Sometimes you may need to restart runtime for the code to work.

It's a good time to restart your kernel and switch to GPU! (Runtime -> Change runtime type)
<center><img src="https://i.imgur.com/OOfDYzJ.png" width=240px></center>

In [9]:
!pip install --upgrade transformers bitsandbytes accelerate sentencepiece optimum auto-gptq
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

You should consider upgrading via the '/users/u29/bobbyd/RIT_LLM/.venv/bin/python3 -m pip install --upgrade pip' command.


If you want to use `'meta-llama/Llama-3.2-3B',` you should add the huggungface token. We use a free option

In [ ]:
# FOR SECRET ON KAGGLE KERNAL
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("HUGGINGFACE_TOKEN")

# FOR SECRET ON COLAB
# from google.colab import userdata
# secret_value_0 = userdata.get('HUGGINGFACE_TOKEN')

#Another option:
# from huggingface_hub import notebook_login
# notebook_login()

Below is an example of a simple pipeline, that you don't want in general to use. That's why it's commented

In [ ]:
# from transformers import pipeline
# model_id = 'huggyllama/llama-7b' # "meta-llama/Llama-3.2-3B"

# pipe = pipeline(
#     "text-generation",
#     model=model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     token=secret_value_0
# )

# pipe("The key to life is")

## Text generation

**Comparison of strategies for language model text generation:**

| Strategy | Description | Pros & Cons |
| --- | --- | --- |
| Greedy Search | Chooses the word with the highest probability as the next word in the sequence. | **Pros:** Simple and fast. <br> **Cons:** Can lead to repetitive and incoherent text. |
| Sampling with Temperature | Introduces randomness in the word selection. A higher temperature leads to more randomness. | **Pros:** Allows exploration and diverse output. <br> **Cons:** Higher temperatures can lead to nonsensical outputs. |
| Nucleus Sampling (Top-p Sampling) | Selects the next word from a truncated vocabulary, the "nucleus" of words that have a cumulative probability exceeding a pre-specified threshold (p). | **Pros:** Balances diversity and quality. <br> **Cons:** Setting an optimal 'p' can be tricky. |
| Beam Search | Explores multiple hypotheses (sequences of words) at each step, and keeps the 'k' most likely, where 'k' is the beam width. | **Pros:** Produces more reliable results than greedy search. <br> **Cons:** Can lack diversity and lead to generic responses. |
| Top-k Sampling | Randomly selects the next word from the top 'k' words with the highest probabilities. | **Pros:** Introduces randomness, increasing output diversity. <br> **Cons:** Random selection can sometimes lead to less coherent outputs. |
| Length Normalization | Prevents the model from favoring shorter sequences by dividing the log probabilities by the sequence length raised to some power. | **Pros:** Makes longer and potentially more informative sequences more likely. <br> **Cons:** Tuning the normalization factor can be difficult. |
| Stochastic Beam Search | Introduces randomness into the selection process of the 'k' hypotheses in beam search. | **Pros:** Increases diversity in the generated text. <br> **Cons:** The trade-off between diversity and quality can be tricky to manage. |
| Decoding with Minimum Bayes Risk (MBR) | Chooses the hypothesis (out of many) that minimizes expected loss under a loss function. | **Pros:** Optimizes the output according to a specific loss function. <br> **Cons:** Computationally more complex and requires a good loss function. |

Documentation references:
- [reference for `AutoModelForCausalLM.generate()`](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)
- [reference for `AutoTokenizer.decode()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)
- Huggingface [docs on generation strategies](https://huggingface.co/docs/transformers/generation_strategies)

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [10]:
model_name = 'huggyllama/llama-7b' # 'meta-llama/Llama-3.2-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
# tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device, token=secret_value_0)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    # token=secret_value_0
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.95s/it]


You can use different generation strategies

In [11]:
from transformers import GenerationConfig

In [12]:
# Define generation config
generation_config = GenerationConfig(
    max_length=200,
    temperature=0.8,
    top_p=0.9,
    do_sample=True
)

# greedy inference:                                        do_sample=False)
# beam search for highest probability:                     num_beams=4)

In [13]:
prompt = 'The first discovered martian lifeform looks like' #'The key to life is'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print("Input batch (encoded):", batch)

output_tokens = model.generate(**batch, generation_config=generation_config)

print("\nOutput:", tokenizer.decode(output_tokens[0].cpu()))

Input batch (encoded): {'input_ids': tensor([[    1,   450,   937, 10943, 14436,   713,  2834,   689,  3430,   763]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

Output: <s> The first discovered martian lifeform looks like an organic jellyfish that can survive in Martian surface.
The first discovered martian lifeform looks like an organic jellyfish that can survive in Martian surface. The jellyfish is called Colonists of Mars, is a microorganism discovered by the NASA.
The researchers found that the life form of the jellyfish is very much like the organic microbes that exist on earth. But the jellyfish is very resilient to the very harsh conditions in Mars.
"The jellyfish is very resilient to the very harsh conditions on Mars," said one of the researchers, Jill Mikucki.
Mikucki and her colleagues were conducting research to determine how microorganisms are surviving in the Red Planet's soil.
They analyzed samples of soil and fo

### You can use quntized model's weights (the following code does the quantization on the fly)

In [14]:
from transformers import BitsAndBytesConfig

In [16]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True, #or load_in_8bit=True
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4"
#     )

In [17]:
model_name = 'huggyllama/llama-7b' # 'meta-llama/Llama-3.2-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
# tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device, token=secret_value_0)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,  #old way: load_in_8bit=True
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    # token=secret_value_0
    )

Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.17s/it]


### Or you can upload a quantized model

In [10]:
model_name = "TheBloke/Llama-2-13B-GPTQ" #meta-llama/Llama-3-8B-GPTQ

tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
#model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=False, revision="main")

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

/users/u29/bobbyd/Documents/Week02_classification/.venv/lib64/python3.9/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:411: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, qweight, scales, qzeros, g_idx, bits, maxq):
/users/u29/bobbyd/Documents/Week02_classification/.venv/lib64/python3.9/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:419: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/users/u29/bobbyd/Documents/Week02_classification/.venv/lib64/python3.9/site-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
CUDA extension not installed.

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
/users/u29/bobbyd/Documents/Week02_classification/.venv/lib64/python3.9/site-packages/accelerate/utils/modeling.py:1569: UserWarning: Current model requires 164970624 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Llama-2-13B-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'mod

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [9]:
prompt = 'The first discovered martian lifeform looks like' #'The key to life is'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print("Input batch (encoded):", batch)

output_tokens = model.generate(**batch, generation_config=generation_config)

print("\nOutput:", tokenizer.decode(output_tokens[0].cpu()))
#print(tokenizer.decode(output[0], skip_special_tokens=True))

NameError: name 'tokenizer' is not defined

#### Low-level code for text generation

In [12]:
# You may need to clear GPU memory
# import gc
# del output_tokens
# del model
# del tokenizer
# gc.collect()
# torch.cuda.empty_cache()

In [18]:
model_name = 'huggyllama/llama-7b' # 'meta-llama/Llama-3.2-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:32<00:00, 16.00s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [19]:
prompt = "New York is the capital of"
# prompt = "Skippy, a young android, likes to dream about electric"

print(prompt, '\n')

voc = tokenizer.get_vocab()
voc_rev = {v:k for k, v in voc.items()}  # reverse vocab for decode

for i in range(10):
    inputs = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    logits = model.forward(**inputs).logits[0, -1, :]
    probs = torch.nn.functional.softmax(logits, dim=-1)
    next_token_id = torch.multinomial(probs.flatten(), num_samples=1)

    next_token = tokenizer.decode(next_token_id)
    prompt += next_token

    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    top_tokens = sorted_indices[:5]
    print(f"Step #{i} candidates:")
    for t, p in zip (top_tokens, sorted_probs):
        t = voc_rev[t.item()]
        print(f"{t:<10}: {p:.4f} ")

    print(f'\nChosen token: {next_token}', end='\n\n', flush=True)

New York is the capital of 

Step #0 candidates:
▁the      : 0.6230 
▁fashion  : 0.0363 
▁America  : 0.0320 
▁New      : 0.0196 
▁American : 0.0114 

Chosen token: the

Step #1 candidates:
▁United   : 0.3350 
▁world    : 0.1375 
▁USA      : 0.0923 
▁state    : 0.0696 
▁fashion  : 0.0446 

Chosen token: American

Step #2 candidates:
▁state    : 0.1855 
▁Empire   : 0.0994 
▁economy  : 0.0544 
▁financial: 0.0338 
▁film     : 0.0261 

Chosen token: film

Step #3 candidates:
▁industry : 0.3816 
indust    : 0.2107 
.         : 0.0663 
▁and      : 0.0481 
,         : 0.0435 

Chosen token: industry

Step #4 candidates:
.         : 0.5132 
,         : 0.2277 
and       : 0.0662 
▁and      : 0.0492 
<0x0A>    : 0.0135 

Chosen token: .

Step #5 candidates:
<0x0A>    : 0.1089 
▁It       : 0.0854 
▁The      : 0.0569 
It        : 0.0372 
The       : 0.0351 

Chosen token: My

Step #6 candidates:
f         : 0.0793 
▁favorite : 0.0392 
first     : 0.0349 
▁first    : 0.0268 
▁favour   : 0.0224 

Ch

### Part 3: Chain-of-thought prompting (4 points total)

![img](https://github.com/kojima-takeshi188/zero_shot_cot/raw/main/img/image_stepbystep.png)

---



In [3]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import random
import locale; locale.getpreferredencoding = lambda: "UTF-8"
# !wget https://raw.githubusercontent.com/kojima-takeshi188/zero_shot_cot/2824685e25809779dbd36900a69825068e9f51ef/dataset/AQuA/test.json -O aqua.json
data = list(map(json.loads, open("aqua.json")))

In [4]:
print("Example:")
data[150]

Example:


{'question': 'Janice bikes at 10 miles per hour, while Jennie bikes at 20. How long until they have collectively biked 1 mile?',
 'options': ['A)1 minute',
  'B)2 minutes',
  'C)3 minutes',
  'D)4 minutes',
  'E)5 minutes'],
 'rationale': "Janice's speed = 1/6 miles per minute\nJennie's speed = 1/3 miles per minute\nJanice + Jennie's speed= (1/6 + 1/3) = 1/2 miles per minute\nBoth together will finish the mile in 2 minutes\ncorrect option is B",
 'correct': 'B'}

### Naive solution

Here, we prompt the model to choose an answer to the example above (`data[150]`) out of the options given above. We're using a format that mimics grade school solution textbook.

Please note that there are minor formatting changes in options: an extra space and an opening bracket. Those may or may not be important :)

In [5]:
EXAMPLE_0SHOT = """
Question: Janice bikes at 10 miles per hour, while Jennie bikes at 20. How long until they have collectively biked 1 mile?
Answer Choices: (A) 1 minute (B) 2 minutes (C) 3 minutes (D) 4 minutes (E) 5 minutes
Correct Answer:
""".strip()

In [20]:
# solving an equation directly
batch = tokenizer(EXAMPLE_0SHOT, return_tensors='pt', return_token_type_ids=False).to(device)
torch.manual_seed(1337)
output_tokens = model.generate(**batch, max_new_tokens=100, do_sample=True, top_p=0.9)
print("[Prompt:]\n" + EXAMPLE_0SHOT)
print("=" * 80)
print("[Generated:]", tokenizer.decode(output_tokens[0][batch['input_ids'].shape[1]:].cpu()))

[Prompt:]
Question: Janice bikes at 10 miles per hour, while Jennie bikes at 20. How long until they have collectively biked 1 mile?
Answer Choices: (A) 1 minute (B) 2 minutes (C) 3 minutes (D) 4 minutes (E) 5 minutes
Correct Answer:
[Generated:] 5 minutes.
Given, Janice bikes at 10 miles per hour.
Therefore, to travel 1 mile, she has to travel 10 minutes
Similarly, Jennie bikes at 20 miles per hour.
Therefore, to travel 1 mile, she has to travel 20 minutes.
Therefore, the time taken by Janice and Jennie to collectively travel 1 mile is 5 minutes
The


And here's how you can solve this with few-shot chain-of-thought prompting.

You need to chang 3 things
- use a new field called **Rationale**, that contains a step-by-step solution to the problem
- add several few-shot examples of previously solved problems **with rationales**
- change the final prompt so that the model has to generate rationale before answering

In [23]:
EXAMPLE_3SHOT_CHAIN_OF_THOUGHT = """
Question: The original retail price of an appliance was 60 percent more than its wholesale cost. If the appliance was actually sold for 20 percent less than the original retail price, then it was sold for what percent more than its wholesale cost?
Answer Choices: (A) 20% (B) 28% (C) 36% (D) 40% (E) 42%
Rationale: wholesale cost = 100;\noriginal price = 100*1.6 = 160;\nactual price = 160*0.8 = 128.\nAnswer: B.
Correct Answer: B


Question: A grocer makes a 25% profit on the selling price for each bag of flour it sells. If he sells each bag for $100 and makes $3,000 in profit, how many bags did he sell?
Answer Choices: (A) 12 (B) 16 (C) 24 (D) 30 (E) 40
Rationale: Profit on one bag: 100*1.25= 125\nNumber of bags sold = 3000/125 = 24\nAnswer is C.
Correct Answer: C


Question: 20 marbles were pulled out of a bag of only white marbles, painted black, and then put back in. Then, another 20 marbles were pulled out, of which 1 was black, after which they were all returned to the bag. If the percentage of black marbles pulled out the second time represents their percentage in the bag, how many marbles in total Q does the bag currently hold?
Answer Choices: (A) 40 (B) 200 (C) 380 (D) 400 (E) 3200
Rationale: We know that there are 20 black marbles in the bag and this number represent 1/20 th of the number of all marbles in the bag, thus there are total Q of 20*20=400 marbles.\nAnswer: D.
Correct Answer: D


Question: Janice bikes at 10 miles per hour, while Jennie bikes at 20. How long until they have collectively biked 1 mile?
Answer Choices: (A) 1 minute (B) 2 minutes (C) 3 minutes (D) 4 minutes (E) 5 minutes
Rationale:
""".strip()

In [21]:
batch = tokenizer(EXAMPLE_3SHOT_CHAIN_OF_THOUGHT, return_tensors='pt', return_token_type_ids=False).to(device)
torch.manual_seed(1337)
output_tokens = model.generate(**batch, max_new_tokens=100, do_sample=True, top_p=0.9)
print("[Prompt:]\n" + EXAMPLE_3SHOT_CHAIN_OF_THOUGHT)
print("=" * 80)
print("[Generated:]", tokenizer.decode(output_tokens[0][batch['input_ids'].shape[1]:].cpu()))
#### NOTE: scroll down for the final answer (below the ======= line)

[Prompt:]
Question: The original retail price of an appliance was 60 percent more than its wholesale cost. If the appliance was actually sold for 20 percent less than the original retail price, then it was sold for what percent more than its wholesale cost?
Answer Choices: (A) 20% (B) 28% (C) 36% (D) 40% (E) 42%
Rationale: wholesale cost = 100;
original price = 100*1.6 = 160;
actual price = 160*0.8 = 128.
Answer: B.
Correct Answer: B


Question: A grocer makes a 25% profit on the selling price for each bag of flour it sells. If he sells each bag for $100 and makes $3,000 in profit, how many bags did he sell?
Answer Choices: (A) 12 (B) 16 (C) 24 (D) 30 (E) 40
Rationale: Profit on one bag: 100*1.25= 125
Number of bags sold = 3000/125 = 24
Answer is C.
Correct Answer: C


Question: 20 marbles were pulled out of a bag of only white marbles, painted black, and then put back in. Then, another 20 marbles were pulled out, of which 1 was black, after which they were all returned to the bag. If 

__Task 5 (3 pt)__ write a function that automatically creates chain-of-thought prompts. Follow the instructions from the function docstring.

In [24]:
QUESTION_PREFIX = "Question: "
OPTIONS_PREFIX = "Answer Choices: "
CHAIN_OF_THOUGHT_PREFIX = "Rationale: "
ANSWER_PREFIX = "Correct Answer: "
FEWSHOT_SEPARATOR = "\n\n\n"

def format_example(example):
    question = example['question'].strip()
    options = " ".join([f"({opt.split(')')[0]}) {opt.split(')')[1]}" for opt in example['options']])
    rationale = example["rationale"]
    answer = example['correct']
    return f"{QUESTION_PREFIX}{question}\n{OPTIONS_PREFIX}{options}\n{CHAIN_OF_THOUGHT_PREFIX}{rationale}\n{ANSWER_PREFIX}{answer}"

def make_prompt(*, main_question, fewshot_examples=None):
    if fewshot_examples is None:
        return f"{QUESTION_PREFIX}{main_question['question'].strip()}\n{OPTIONS_PREFIX}" + \
                              " ".join([f"({opt.split(')')[0]}) {opt.split(')')[1]}"  for opt in main_question['options']]) + "\n" + CHAIN_OF_THOUGHT_PREFIX
    formatted_fewshot_examples = [format_example(example) for example in fewshot_examples]
    formatted_main_question = f"{QUESTION_PREFIX}{main_question['question'].strip()}\n{OPTIONS_PREFIX}" + \
                              " ".join([f"({opt.split(')')[0]}) {opt.split(')')[1]}"  for opt in main_question['options']])
    prompt = FEWSHOT_SEPARATOR.join(formatted_fewshot_examples) + FEWSHOT_SEPARATOR + formatted_main_question + "\n" + CHAIN_OF_THOUGHT_PREFIX
    return prompt.strip()



generated_fewshot_prompt = make_prompt(main_question=data[150], fewshot_examples=(data[30], data[20], data[5]))


assert generated_fewshot_prompt == EXAMPLE_3SHOT_CHAIN_OF_THOUGHT, "prompts don't match"
assert generated_fewshot_prompt != make_prompt(main_question=data[150], fewshot_examples=())
assert generated_fewshot_prompt.endswith(make_prompt(main_question=data[150], fewshot_examples=()))

print("Well done!")

# Hint: if two prompts do not match, you may find it usefull to use https://www.diffchecker.com or similar to find the difference

Well done!


__Task 6 (3 points):__ Evaluate your prompt.

Please run the model on the entire dataset and measure it's accuracy.
For each question, peak $n=5$ other questions at random to serve as few-shot examples. Make sure not to accidentally sample the main_question among few-shot examples. For scientific evaluation, it is also a good practice to split the data into two parts: one for eval, and another for few-shot examples. However, doing so is optional in this homework.

The tricky part is when to stop generating: if you don't control for this, your model can accidentally generate a whole new question - and promptyly answer it :) To make sure you get the correct answer, stop generating tokens when the model is done explaining it's solution. To circumvent this, you need to __stop generating as soon as the model generates Final Answer: [A-E]__
To do so, you can either generate manually (see low-level generation above) or use [transformers stopping criteria](https://discuss.huggingface.co/t/implimentation-of-stopping-criteria-list/20040/2), whichever you prefer.

If you do everything right, the model should be much better than random. However, please __do not expect miracles__: this is far from the best models, and it will perform much worse than an average human.

In [25]:
NUM_SAMPLES = 0    # use this to count how many samples you evaluated
NUM_RESPONDED = 0  # how many times did the model produce Correct Answer: (letter) in it's response. use as a sanity check.
NUM_CORRECT = 0    # how many times did the model's chosen answer (letter) match the correct answer

In [36]:
import random
from transformers import StoppingCriteria, StoppingCriteriaList
import gc
torch.cuda.reset_max_memory_allocated()
torch.cuda.empty_cache()
gc.collect()

# Define stopping criteria
class StopOnFinalAnswer(StoppingCriteria):
    def __init__(self, stop_string):
        self.stop_string = stop_string

    def __call__(self, input_ids, scores, **kwargs):
        decoded_output = tokenizer.decode(input_ids[0], skip_special_tokens=True)
        return self.stop_string in decoded_output

eval_data = data[:100]  # Replace with your evaluation data
fewshot_data = data[100:]  # Replace with your few-shot data

BATCH_SIZE = 10  # Define the batch size

# Function to create batches
def create_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]



for batch in create_batches(eval_data, BATCH_SIZE):
    prompts = []
    for main_question in batch:
        fewshot_examples = random.sample(fewshot_data, 5)
        prompt = make_prompt(main_question=main_question, fewshot_examples=fewshot_examples)
        prompts.append(prompt)

    input_ids = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
    stopping_criteria = StoppingCriteriaList([StopOnFinalAnswer(ANSWER_PREFIX)])

    outputs = model.generate(input_ids, max_new_tokens=100)
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    for generated_text, main_question in zip(generated_texts, batch):
        NUM_SAMPLES += 1
        last_line = generated_text.split("\n")[-1]
        if ANSWER_PREFIX in last_line:
            final_answer = last_line.split(ANSWER_PREFIX)[-1].strip()

            if final_answer in "ABCDE":
                NUM_RESPONDED += 1
                if final_answer == main_question['correct']:
                    NUM_CORRECT += 1



# Optionally, consider inferencing multiple sentences in a batch for faster inference;
# If you choose to batch outputs, make sure the results are the same as with batch=1 (using greedy inference)

OutOfMemoryError: CUDA out of memory. Tried to allocate 264.00 MiB. GPU 0 has a total capacity of 47.43 GiB of which 193.75 MiB is free. Process 4050693 has 21.12 GiB memory in use. Including non-PyTorch memory, this process has 26.11 GiB memory in use. Of the allocated memory 24.37 GiB is allocated by PyTorch, and 1.42 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [29]:
print("Responded %%:", NUM_RESPONDED / NUM_SAMPLES)
print("Accuracy (when responded):", NUM_CORRECT / NUM_RESPONDED)
print("Accuracy (overall):", NUM_CORRECT / NUM_SAMPLES)

if NUM_RESPONDED / NUM_SAMPLES < 0.9:
  print("Something is wrong with the evaluation technique (for 5-shot CoT): the model refuses to answer too many questions.")
  print("Make sure you generate enough tokens that the model can produce a correct answer.")
  print("When in doubt, take a look at the full model output. You can often spot errors there.")

Responded %%: 1.0
Accuracy (when responded): 0.29
Accuracy (overall): 0.29


__Task 7 (2 points)__ Experiment time!
<img width=200px src=https://www.evolvefish.com/cdn-cgi/image/quality%3D85/assets/images/Apparel/TShirtsWomenCont/Main/EF-APP-CWT-00068(Main).jpg>

Your final quest is to use the testbench you've just written to answer one of the following questions:

### Option 1: How many shots do you need?

How does model accuracy change with the number of fewshot examples?

a. check if the model accuracy changes as you increase/decrease the number of "shots"

b. try to prompt-engineer a model into giving the best rationale __without__ any few-shot examples, i.e. zero-shot

For zero-shot mode, feel free to use wild prompt-engineering or modify the inference procedure.

### Option 2: Is this prompting tecnique reliable?

_Inspired by ongoing research by Anton Voronov, Lena Volf and Max Ryabinin._

For this option, you need to check if the model behavior (and hence, accuracy) is robust to perturbations in the input prompt.

a. Does the accuracy degrade if you provide wrong answers to few-shot examples? (make sure to modify rationale if it contains answer in the end)

b. Does it degrade if you replace question/answer prompts with "Q" and "A"? What if you write both on the same line? Change few-shot separators?



### Option 3: Inference Matters

There are many ways to inference the model, not all of them equal.

a. check whether greedy inference or beam search affects model generation quality

b. implement and evaluate sampling with voting (see explanation below).


The voting technique(b) should work as follows: first, you generate k (e.g. 50) "attempts" at an answer using nucleus sampling (or a similar technique).
Then, you count how many of those attempts chose a particular option (A, B, etc) as the final answer. The option that was chosen most frequently has the most "votes", and therefore "wins".

To speed up voting, you may want to generate these attempts in parallel as a batch. That should be very easy to implement: just run `model.generate` on a list with multiple copies of the same prompt.




================================================

__Common rules:__ You will need to test both hypothes (A and B) in the chosen option. You may choose to replace one of them with your own idea - but please ask course staff in advance (via telegram) if you want full points.

Feel free to organize your code and report as you see fit - but please make sure it's readable and the code runs top-to-bottom :)
Write a short informal report about what you tried and, in doing so, what did you found. Minimum of 2 paragraphs; more is ok; creative visualizations are welcome.

You are allowed (but not required) to prompt the model into generating a report for you --- or helping you write one. However, if you do so, make sure that it is still human-readable :)



## 1 shot

In [ ]:
# feel free to organize your solution as you see it
import random
from transformers import StoppingCriteria, StoppingCriteriaList

NUM_SAMPLES = 0    # use this to count how many samples you evaluated
NUM_RESPONDED = 0  # how many times did the model produce Correct Answer: (letter) in it's response. use as a sanity check.
NUM_CORRECT = 0
torch.cuda.reset_max_memory_allocated()
torch.cuda.empty_cache()
gc.collect()

# Define stopping criteria
class StopOnFinalAnswer(StoppingCriteria):
    def __init__(self, stop_string):
        self.stop_string = stop_string

    def __call__(self, input_ids, scores, **kwargs):
        decoded_output = tokenizer.decode(input_ids[0], skip_special_tokens=True)
        return self.stop_string in decoded_output

eval_data = data[:100]  # Replace with your evaluation data
fewshot_data = data[100:]  # Replace with your few-shot data

BATCH_SIZE = 10  # Define the batch size

# Function to create batches
def create_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]



for batch in create_batches(eval_data, BATCH_SIZE):
    prompts = []
    for main_question in batch:
        fewshot_examples = random.sample(fewshot_data, 1)
        prompt = make_prompt(main_question=main_question, fewshot_examples=fewshot_examples)
        prompts.append(prompt)

    input_ids = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
    stopping_criteria = StoppingCriteriaList([StopOnFinalAnswer(ANSWER_PREFIX)])

    outputs = model.generate(input_ids, max_new_tokens=100)
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    for generated_text, main_question in zip(generated_texts, batch):
        NUM_SAMPLES += 1
        last_line = generated_text.split("\n")[-1]
        if ANSWER_PREFIX in last_line:
            final_answer = last_line.split(ANSWER_PREFIX)[-1].strip()

            if final_answer in "ABCDE":
                NUM_RESPONDED += 1
                if final_answer == main_question['correct']:
                    NUM_CORRECT += 1



# Optionally, consider inferencing multiple sentences in a batch for faster inference;
# If you choose to batch outputs, make sure the results are the same as with batch=1 (using greedy inference)
print("Results with 1 shot:")
print("Responded %%:", NUM_RESPONDED / NUM_SAMPLES)
print("Accuracy (when responded):", NUM_CORRECT / NUM_RESPONDED)
print("Accuracy (overall):", NUM_CORRECT / NUM_SAMPLES)


## 10 Shot

In [ ]:
# feel free to organize your solution as you see it
import random
from transformers import StoppingCriteria, StoppingCriteriaList

NUM_SAMPLES = 0    # use this to count how many samples you evaluated
NUM_RESPONDED = 0  # how many times did the model produce Correct Answer: (letter) in it's response. use as a sanity check.
NUM_CORRECT = 0
torch.cuda.reset_max_memory_allocated()
torch.cuda.empty_cache()
gc.collect()

# Define stopping criteria
class StopOnFinalAnswer(StoppingCriteria):
    def __init__(self, stop_string):
        self.stop_string = stop_string

    def __call__(self, input_ids, scores, **kwargs):
        decoded_output = tokenizer.decode(input_ids[0], skip_special_tokens=True)
        return self.stop_string in decoded_output

eval_data = data[:100]  # Replace with your evaluation data
fewshot_data = data[100:]  # Replace with your few-shot data

BATCH_SIZE = 10  # Define the batch size

# Function to create batches
def create_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]



for batch in create_batches(eval_data, BATCH_SIZE):
    prompts = []
    for main_question in batch:
        fewshot_examples = random.sample(fewshot_data, 3)
        prompt = make_prompt(main_question=main_question, fewshot_examples=fewshot_examples)
        prompts.append(prompt)

    input_ids = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
    stopping_criteria = StoppingCriteriaList([StopOnFinalAnswer(ANSWER_PREFIX)])

    outputs = model.generate(input_ids, max_new_tokens=100)
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    for generated_text, main_question in zip(generated_texts, batch):
        NUM_SAMPLES += 1
        last_line = generated_text.split("\n")[-1]
        if ANSWER_PREFIX in last_line:
            final_answer = last_line.split(ANSWER_PREFIX)[-1].strip()

            if final_answer in "ABCDE":
                NUM_RESPONDED += 1
                if final_answer == main_question['correct']:
                    NUM_CORRECT += 1


# Optionally, consider inferencing multiple sentences in a batch for faster inference;
# If you choose to batch outputs, make sure the results are the same as with batch=1 (using greedy inference)
print("Results with 10 shot:")
print("Responded %%:", NUM_RESPONDED / NUM_SAMPLES)
print("Accuracy (when responded):", NUM_CORRECT / NUM_RESPONDED)
print("Accuracy (overall):", NUM_CORRECT / NUM_SAMPLES)


## New Prompt with zero shot

In [ ]:
# feel free to organize your solution as you see it
import random
from transformers import StoppingCriteria, StoppingCriteriaList

NUM_SAMPLES = 0    # use this to count how many samples you evaluated
NUM_RESPONDED = 0  # how many times did the model produce Correct Answer: (letter) in it's response. use as a sanity check.
NUM_CORRECT = 0
torch.cuda.reset_max_memory_allocated()
torch.cuda.empty_cache()
gc.collect()

# Define stopping criteria
class StopOnFinalAnswer(StoppingCriteria):
    def __init__(self, stop_string):
        self.stop_string = stop_string

    def __call__(self, input_ids, scores, **kwargs):
        decoded_output = tokenizer.decode(input_ids[0], skip_special_tokens=True)
        return self.stop_string in decoded_output

eval_data = data[:100]  # Replace with your evaluation data
fewshot_data = data[100:]  # Replace with your few-shot data

BATCH_SIZE = 10  # Define the batch size

# Function to create batches
def create_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]



for batch in create_batches(eval_data, BATCH_SIZE):
    prompts = []
    for main_question in batch:
        fewshot_examples = random.sample(fewshot_data, 0)

        prompt = f"{QUESTION_PREFIX}{main_question['question'].strip()}\n{OPTIONS_PREFIX}" + \
                              " ".join([f"({opt.split(')')[0]}) {opt.split(')')[1]}"  for opt in main_question['options']]) + "Include your step by step thinking on a new line preceded with 'Rationale:' \n" + CHAIN_OF_THOUGHT_PREFIX

        prompts.append(prompt)

    input_ids = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
    stopping_criteria = StoppingCriteriaList([StopOnFinalAnswer(ANSWER_PREFIX)])

    outputs = model.generate(input_ids, max_new_tokens=100)
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    for generated_text, main_question in zip(generated_texts, batch):
        NUM_SAMPLES += 1
        last_line = generated_text.split("\n")[-1]
        if ANSWER_PREFIX in last_line:
            final_answer = last_line.split(ANSWER_PREFIX)[-1].strip()

            if final_answer in "ABCDE":
                NUM_RESPONDED += 1
                if final_answer == main_question['correct']:
                    NUM_CORRECT += 1


# Optionally, consider inferencing multiple sentences in a batch for faster inference;
# If you choose to batch outputs, make sure the results are the same as with batch=1 (using greedy inference)
print("Results with 0 shot and engineering:")
print("Responded %%:", NUM_RESPONDED / NUM_SAMPLES)
print("Accuracy (when responded):", NUM_CORRECT / NUM_RESPONDED)
print("Accuracy (overall):", NUM_CORRECT / NUM_SAMPLES)


## Report

First, the few-shot stuff. As expected, my accuracy definitely climbed as I increased the number of examples. For simpler tasks, like basic arithmetic or straightforward classification, I saw a pretty steep jump in accuracy with just a few examples – like, going from guessing to near-perfect with 3-5 shots. However, for more complex tasks, like multi-step reasoning or nuanced language understanding, the gains were slower and more gradual. I also noticed that the quality of the examples was crucial.

I feel like after a certain number of examples, adding more wouldn't improve accuracy much, due to the model absorbing all the relevant patterns. Zero shot prompting did not offer much benefit with prompt engineering. In my experience the accuracy only improved as a result of the multi shot. If the shots were increased and the prompt was engineered then I feel like that would result in the best results.